# Imports

In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from collections import defaultdict

import common_functions
import numpy as np
import pandas as pd
import utils
from scipy import stats

# Aim of this notebook  
* Calculate the Estimated Daily Intake values based on the wristband and dust measurements


Based on the concentrations detailed, we estimate exposure levels through the ingestion of dust and wristband. The estimated daily intake (EDI) in milligrams per kilogram of body weight per day (mg/kg bw/day) was determined using a methodology based on the general approach described by McGrath et al., 2022. The EDI is calculated using the following formula:

EDI = (Concentration × Ingestion × Fraction) / Body weight

Here, "concentration" represents the concentrations of quantified compounds."Ingestion" pertains to dust ingestion rates of 20 and 60 mg/day for adults and toddlers in the 50th percentile exposure scenario, and 50 and 100 mg/day for adults and toddlers in the 95th percentile exposure scenario, as specified by USEPA, 2017a."Fraction" refers to the fraction of time individuals spend at the workplace (0.33, 8 hours of work divided by 24 hours), based on research by Klepeis et al., 2001, and Poma et al., 2020. "Body weight" is given in the serum dataframe, expressed in kg unit.

It's worth noting that we assumed 100% bioaccessibility for each compound, providing a conservative estimate of internal exposure, as per Christia et al., 2021.

In [2]:
DUST_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath("dust.parquet.gzip")

SERUM_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath(
    "HBM4EU_E-waste_template_V3_all_data_INTERIM.parquet.gzip"
)

WRISTBAND_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath(
    "wristband.parquet.gzip"
)

dust = pd.read_parquet(DUST_DATA_PATH).groupby("group").mean()
serum = pd.read_parquet(SERUM_DATA_PATH)
wristband = pd.read_parquet(WRISTBAND_DATA_PATH)

## EDI
### Based on the dust measurements
#### 50th percentile

To calculate the 50th percentile exposure scenario, we use dust ingestion rates of 20 mg/day. To get the unit of **mg/kg/day** we will use 0.02 g/day intake and divide the final values by 1000, since the original dust measurements are in µg/g dust.

In [4]:
dust_EDI_p50 = (
    serum.query("main_category == 'Worker'")[["companyID", "weight"]]
    .pipe(
        lambda df: df.assign(
            **{
                col
                + "_dust": lambda df, col=col: df.companyID.map(
                    dict(zip(dust.index, dust[col]))
                )
                for col in dust.columns
            }
        )
    )
    .pipe(
        lambda df: df.assign(
            **{
                col
                + "_EDI_p50": lambda df, col=col: (df[col] * 0.02 * (8 / 24))
                / df["weight"]
                for col in df.loc[:, "PCB 28_dust":"BDE 209_dust"].columns
            }
        )
    )
    .loc[:, "PCB 28_dust_EDI_p50":]
    .describe()
    .transpose()
    .loc[:, ["count", "25%", "50%", "75%"]]
    .div(1000)
)

dust_EDI_p50

,count,25%,50%,75%
PCB 28_dust_EDI_p50,0.103,4.537037e-09,2.050654e-08,5.080836e-08
PCB 52_dust_EDI_p50,0.103,4.357298e-09,1.486111e-08,4.695767e-08
PCB 101_dust_EDI_p50,0.103,5.998076e-09,1.092279e-08,3.000000e-08
PCB 118_dust_EDI_p50,0.103,5.860189e-09,1.067169e-08,3.017857e-08
PCB 138_dust_EDI_p50,0.103,4.901961e-09,6.320988e-09,2.063785e-08
PCB 153_dust_EDI_p50,0.103,5.791246e-09,9.375000e-09,1.603213e-08
PCB 180_dust_EDI_p50,0.103,3.921569e-09,5.128205e-09,1.769288e-08
BDE 28_dust_EDI_p50,0.103,3.898769e-09,6.153846e-09,1.006757e-08
BDE 47_dust_EDI_p50,0.103,5.089944e-09,1.302682e-08,4.979562e-08
BDE 99_dust_EDI_p50,0.103,9.078483e-09,2.297297e-08,4.979562e-08


#### 95th percentile
To calculate the 95th percentile exposure scenario, we use dust ingestion rates of 60 mg/day, that is 0.06 g/day.

In [5]:
dust_EDI_p95 = (
    serum.query("main_category == 'Worker'")[["companyID", "weight"]]
    .pipe(
        lambda df: df.assign(
            **{
                col
                + "_dust": lambda df, col=col: df.companyID.map(
                    dict(zip(dust.index, dust[col]))
                )
                for col in dust.columns
            }
        )
    )
    .pipe(
        lambda df: df.assign(
            **{
                col
                + "_EDI_p95": lambda df, col=col: (df[col] * 0.06 * (8 / 24))
                / df["weight"]
                for col in df.loc[:, "PCB 28_dust":"BDE 209_dust"].columns
            }
        )
    )
    .loc[:, "PCB 28_dust_EDI_p95":]
    .describe()
    .transpose()
    .loc[:, ["count", "25%", "50%", "75%"]]
    .div(1000)
)

dust_EDI_p95

,count,25%,50%,75%
PCB 28_dust_EDI_p95,0.103,1.361111e-08,6.151961e-08,1.524251e-07
PCB 52_dust_EDI_p95,0.103,1.307190e-08,4.458333e-08,1.408730e-07
PCB 101_dust_EDI_p95,0.103,1.799423e-08,3.276836e-08,9.000000e-08
PCB 118_dust_EDI_p95,0.103,1.758057e-08,3.201507e-08,9.053571e-08
PCB 138_dust_EDI_p95,0.103,1.470588e-08,1.896296e-08,6.191354e-08
PCB 153_dust_EDI_p95,0.103,1.737374e-08,2.812500e-08,4.809639e-08
PCB 180_dust_EDI_p95,0.103,1.176471e-08,1.538462e-08,5.307863e-08
BDE 28_dust_EDI_p95,0.103,1.169631e-08,1.846154e-08,3.020270e-08
BDE 47_dust_EDI_p95,0.103,1.526983e-08,3.908046e-08,1.493868e-07
BDE 99_dust_EDI_p95,0.103,2.723545e-08,6.891892e-08,1.493868e-07


### Based on the wristband measurements
#### 50th percentile

To get the unit of mg/kg/day we will use 0.02 g/day intake and divide the final values by 1000_000, since the original wristband measurements are in ng/g wristband.

In [6]:
wristband_EDI_p50 = (
    serum[["weight"]]
    .merge(wristband, left_index=True, right_on="ID")
    .drop(columns=["ID", "main_category", "company_ID"])
    .pipe(
        lambda df: df.assign(
            **{
                col
                + "_EDI_p50": lambda df, col=col: (df[col] * 0.02 * (8 / 24))
                / df["weight"]
                for col in df.loc[:, "PCB 101":"BDE 209"].columns
            }
        )
    )
    .loc[:, "PCB 101_EDI_p50":]
    .describe()
    .transpose()
    .loc[:, ["count", "25%", "50%", "75%"]]
    .div(1000_000)
)
wristband_EDI_p50

,count,25%,50%,75%
PCB 101_EDI_p50,0.000079,4.166667e-11,6.014662e-10,1.465132e-09
PCB 118_EDI_p50,0.000079,1.456436e-10,3.419923e-10,9.940768e-10
PCB 153_EDI_p50,0.000079,1.200631e-10,2.367025e-10,5.835606e-10
PCB 138_EDI_p50,0.000079,1.521340e-10,3.543171e-10,1.229687e-09
PCB 180_EDI_p50,0.000079,3.788368e-11,9.466282e-11,2.440699e-10
BDE 28_EDI_p50,0.000079,3.508772e-11,3.921569e-11,4.901961e-11
BDE 47_EDI_p50,0.000079,3.921569e-11,2.174448e-10,1.170292e-09
BDE 100_EDI_p50,0.000079,3.546501e-11,4.166667e-11,2.236451e-10
BDE 99_EDI_p50,0.000079,3.508772e-11,4.166667e-11,1.225952e-09
BDE 154_EDI_p50,0.000079,3.508772e-11,3.921569e-11,7.312330e-11


#### 95th percentile

To get the unit of mg/kg/day we will use 0.06 g/day intake and divide the final values by 1000_000, since the original wristband measurements are in ng/g wristband.

In [7]:
wristband_EDI_p95 = (
    serum[["weight"]]
    .merge(wristband, left_index=True, right_on="ID")
    .drop(columns=["ID", "main_category", "company_ID"])
    .pipe(
        lambda df: df.assign(
            **{
                col
                + "_EDI_p50": lambda df, col=col: (df[col] * 0.06 * (8 / 24))
                / df["weight"]
                for col in df.loc[:, "PCB 101":"BDE 209"].columns
            }
        )
    )
    .loc[:, "PCB 101_EDI_p50":]
    .describe()
    .transpose()
    .loc[:, ["count", "25%", "50%", "75%"]]
    .div(1000_000)
)
wristband_EDI_p95

,count,25%,50%,75%
PCB 101_EDI_p50,0.000079,1.250000e-10,1.804399e-09,4.395397e-09
PCB 118_EDI_p50,0.000079,4.369307e-10,1.025977e-09,2.982230e-09
PCB 153_EDI_p50,0.000079,3.601894e-10,7.101075e-10,1.750682e-09
PCB 138_EDI_p50,0.000079,4.564021e-10,1.062951e-09,3.689062e-09
PCB 180_EDI_p50,0.000079,1.136510e-10,2.839885e-10,7.322098e-10
BDE 28_EDI_p50,0.000079,1.052632e-10,1.176471e-10,1.470588e-10
BDE 47_EDI_p50,0.000079,1.176471e-10,6.523344e-10,3.510876e-09
BDE 100_EDI_p50,0.000079,1.063950e-10,1.250000e-10,6.709354e-10
BDE 99_EDI_p50,0.000079,1.052632e-10,1.250000e-10,3.677855e-09
BDE 154_EDI_p50,0.000079,1.052632e-10,1.176471e-10,2.193699e-10
